# Multicriterio

Implementación de AHP para base de datos con alternativas y criterios tanto numéricos como categóricos.

In [1]:
import numpy as np
import pandas as pd

xls=pd.ExcelFile(r"C:\Users\marti\OneDrive\Documentos\personal\Universidad\Septimo_semestre\Opti_multiobjective\Multicriterio\imdbmoviesdata.xlsx")
datos=pd.read_excel(xls,"tmdb_movies_data")
tabla_AHP=pd.read_excel(xls,"AHP")
tabla_AHP_gen=pd.read_excel(xls,"AHP_genero")
#tabla_AHP_time=pd.read_excel(xls,"AHP_duracion")


xls2=pd.ExcelFile(r"C:\Users\marti\OneDrive\Documentos\personal\Universidad\Septimo_semestre\Opti_multiobjective\Multicriterio\Taller AHP.xlsx")
datos2=pd.read_excel(xls2,"Datos")
t_AHP=pd.read_excel(xls2,"AHP")
AHP_tipo=pd.read_excel(xls2,"AHP_tipo")
#[65,90,120,180,340]


In [2]:
def prepos_data(datos:pd.DataFrame,vect_max_min):
    """Recibe:
    datos: base de datos
    vect_max_min: vector con tantas entradas como variables numéricas y en orden de aparición en la base de datos.
    1 si es de maximización, 0 si es de minimización.
    Retorna:
    Base de datos normalizada tomando en cuenta objetivo (min o max)"""
    datos_num=datos.select_dtypes(include="number")
    #print(datos_num.head())
    if len(datos_num.columns)!=len(vect_max_min):
       raise Exception(f"Dimensiones no coincidentes datos numéricos con {len(datos_num.columns)} columnas y el vector con {len(vect_max_min)} entradas")
    for i in range(len(vect_max_min)):
        if vect_max_min[i]==1:
            datos_num.iloc[:,i]=(datos_num.iloc[:,i]-datos_num.iloc[:,i].min())/(datos_num.iloc[:,i].max()-datos_num.iloc[:,i].min())
        else: 
            datos_num.iloc[:,i]=(datos_num.iloc[:,i].max()-datos_num.iloc[:,i])/(datos_num.iloc[:,i].max()-datos_num.iloc[:,i].min())
    datos[datos_num.columns]=datos_num #Asignación sobre el mismo objeto DataFrame

def calc_ponder(matr:np.array,tol:float,categorica=False):
    """Recibe:
    matr: matriz nxn de comparación AHP dos a dos recíproca.
    tol: tolerancia de variación entre vectores suma normalizados.
    Retorna:
    vect_pond: vector ponderación. Eleva error si no se cumple consistencia"""
    n,_=matr.shape
    epsilon=np.array([tol+1])
    i=2
    prev_v_norm=matr.sum(axis=1)/matr.sum()
    while (epsilon>=tol).any():
        if i>6:
            raise Exception("Matriz no consistente por iteraciones")
        A_n=np.linalg.matrix_power(matr,i)#Eleva error si no es cuadrada
        v_norm=A_n.sum(axis=1)/A_n.sum() #vector de suma de filas normalizado
        epsilon=max(v_norm-prev_v_norm)
        prev_v_norm=v_norm
    #print("\n v_norm", v_norm)
    if not categorica:
        #Verificación consistencia
        v1=np.matmul(matr,v_norm)
        v1_perc=np.divide(v1,v_norm)
        #print("\n v1,v1_perc\n",v1,v1_perc)
        lambda_max=np.mean(v1_perc)
        #print(lambda_max)
        indice_RI={1:0,3:0.58,4:0.89,5:1.11,6:1.24,7:1.32,8:1.4,9:1.5,10:1.49}
        RI=indice_RI[n]
        CI=(lambda_max-n)/(n-1)
        CR=CI/RI
        if CR>0.1:
            raise Exception("Matriz no consistente por prueba")
    return v_norm

def categ_AHP(tabla_AHP_cat:pd.DataFrame)->dict:
    """Recibe:
    tabla_AHP_cat: tabla comparativa sobre los posibles valores de una variable (criterio) categórica
    Retorna:
    Diccionario con valores:ponderaciónAHP"""
    
    valores=list(tabla_AHP_cat.iloc[:,0])
    matriz=tabla_AHP_cat[valores].values
    vect_pond=calc_ponder(matriz,0.05,categorica=True)
    dicc_valor_ponderacion=dict(zip(valores,vect_pond))
    return dicc_valor_ponderacion

    
def AHP(datos,tabla_AHP,vect_max_min,n_result,*tablas_AHP_cat):
    """Recibe:
    datos: base de datos de alternativas
    tabla_AHP: matriz recíproca sobre cada criterio.
    vect_max_min: vector con tantas entradas como variables numéricas y especificando con orden respectivo
    a la base de datos: 0 si es minimización, 1 si es maximización
    n_result: número deseado de mejores alternativas.
    tablas_AHP_cat: tablas AHP para variables categóricas"""
    prepos_data(datos,vect_max_min) #Altera el objeto datos
    valores_vab=list(tabla_AHP.iloc[:,0])
    
    matriz_AHP=tabla_AHP[valores_vab].values
    v_ponderac_criter=calc_ponder(matriz_AHP,0.05)
    
    for tabla_cat in tablas_AHP_cat:
        criterio=tabla_cat.columns[0]#La tabla debe tener como primer elemento 
        #del encabezado el nombre del criterio corespondiente y como primera columna, los valores que puede tomar
        dicc_valorac=categ_AHP(tabla_cat)
        datos[criterio]=datos[criterio].replace(dicc_valorac)
    #print("sum v_pond",np.sum(v_ponderac_criter))
    rank=np.matmul(datos[valores_vab].values,v_ponderac_criter)
    
    #print("rangos",rank)
    indices=np.argsort(rank)[:n_result]#rank.max()==rank
    return datos.iloc[indices,0]
    


# def naive(datos:pd.DataFrame,vect_max_min_total,dicc):
#     """
#     vect_max_min: vector de 0 y 1 pero tantos como variables haya (categs y numéricas)
#     Sólo una vab categórica"""
#     datos_num=datos.select_dtypes(include="number")
#     categ_cols=set(datos.columns)-set(datos_num.columns)-{datos.columns[0]}
#     #for i in categ_cols:
#     print(categ_cols)
#     prepos_data(datos)



In [3]:
#naive
#Creo diccionario con pesos
dicc_naive={"Drama":5,"Comedy":8,"Action":5,"Adventure":6,"Horror":1,"Crime":2,"Thriller":6,"Fantasy":4,"Animation":3,"Science Fiction":6,"Romance":3,"Familiy":2,"Mystery":3,"Music":3,"War":4,"Western":6}
suma_pesos=np.sum([i for i in dicc_naive.values()])
#Pesos normalizados
dicc_naive={key:value/suma_pesos for key,value in dicc_naive.items()}
#Matriz que usaremos para el proceso naive
datos_naive=pd.DataFrame.copy(datos)
datos_naive["Género"]=datos_naive["Género"].replace(dicc_naive)#.astype(float)
datos_naive["Género"].value_counts()
#Vector de min_max debe incluir todas las variables
prepos_data(datos_naive,[1,1,1,0,1,1,1,1])
valores_vab=list(datos_naive.columns[1:])
matriz_datos=datos_naive[valores_vab].values
vect_pond_naive=np.array([4,3,5,7,7,6,6,4])
ranking=np.matmul(matriz_datos,vect_pond_naive)
indices=np.argsort(ranking)[:10]#rank.max()==rank
datos.iloc[indices,0]


3187                            Exorcist II: The Heretic
1948                                              Carlos
3003                                              Jaws 3
3175                                          FearDotCom
3061                  Halloween III: Season of the Witch
3092                                            C.H.U.D.
3173                                     Poltergeist III
3013                                   House of the Dead
3168    Friday the 13th Part VIII: Jason Takes Manhattan
2960                            The Pit and the Pendulum
Name: Nombre, dtype: object

In [4]:
resultado=list(AHP(datos,tabla_AHP,[1,1,1,0,1,1,1],10,tabla_AHP_gen))
resultado

['FearDotCom',
 'Ghoulies',
 'House of the Dead',
 'Exorcist II: The Heretic',
 'Jaws 3',
 'C.H.U.D.',
 'From Justin to Kelly',
 'The Next Best Thing',
 'Poltergeist III',
 'Texas Chainsaw Massacre: The Next Generation']